In [ ]:
import os
import sys
import glob
import shutil
import pandas as pd
import numpy as np
from ipywidgets import interact
from copy import deepcopy
import matplotlib.pyplot as plt
print('OK')

In [ ]:
sys.path.append('..')

In [ ]:
from dataset.dataset.dsindex import FilesIndex
from preprocessing.ct_masked_batch import CTImagesMaskedBatch as CTIMB

### Global index

In [ ]:
DIR_LUNA = '/notebooks/data/MRT/luna/s*/*.mhd'
DIR_DUMP = '/notebooks/Koryagin/Batch_dev/dir_for_dumps/'

In [ ]:
if os.path.exists(DIR_DUMP):
    shutil.rmtree(DIR_DUMP)

In [ ]:
ind = FilesIndex(path=DIR_LUNA, no_ext=True)

### Create batch by subindexing

In [ ]:
batch_masked = CTIMB(ind.create_subset(ind.index[[714, 297, 299, 672]]))
batch_masked_2 = CTIMB(ind.create_subset(ind.index[[714, 297, 299, 672]]))

### Test actions

* check load

In [ ]:
batch_masked.load(fmt='raw')
batch_masked_2.load(fmt='raw')

In [ ]:
batch_copy = deepcopy(batch_masked)

* check load mask

In [ ]:
nodules = pd.read_csv('/notebooks/data/MRT/luna/CSVFILES/annotations.csv')
nodules.head(2)

In [ ]:
batch_masked = batch_masked.load_mask(nodules_df=nodules)

* check resize

In [ ]:
batch_masked = batch_masked.resize(num_slices_new=128, num_x_new=256, num_y_new=256,
                                   num_threads=20)

In [ ]:
batch_masked_2 = batch_masked_2.resize(num_slices_new=128, num_x_new=256, num_y_new=256,
                                       num_threads=20)

In [ ]:
batch_masked_2 = batch_masked_2.load_mask(nodules_df=nodules)

In [ ]:
batch_masked_2.indices

In [ ]:
batch_masked.indices

### Compare

In [ ]:
def plot_slices(height, num_pers, *batches):
    fig, axes = plt.subplots(1, len(batches), figsize = (14, len(batches)*8))
    com_args = dict(clim=(-1200, 300), cmap=(plt.cm.gray))
    
    for i in range(len(batches)):
        if batches[i][1] == 'scan':
            axes[i].imshow(batches[i][0].get_axial_slice(num_pers, height)[0], **com_args)
        elif batches[i][1] == 'mask':
            axes[i].imshow(batches[i][0].get_axial_slice(num_pers, height)[1], cmap=plt.cm.gray)
    plt.show()

In [ ]:
interact(lambda height: plot_slices(height, 1, [batch_masked, 'scan'],
                                    [batch_masked, 'mask']
                                    ,[batch_masked_2, 'mask']
                                   ), height=(0.01, 0.99, 0.01))

